<a href="https://colab.research.google.com/github/pratap977/CS6910/blob/main/CNN_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install split-folders

In [2]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 8.1 MB/s 
     |████████████████████████████████| 144 kB 79.5 MB/s 
     |████████████████████████████████| 181 kB 67.4 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=327ad4a348e8db87300a96d69b0a5cf745d6d0a7f456c16512b2cc74c57bedb8
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image as im
import math as mh
from matplotlib import pyplot
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Activation,Dense,Flatten
from keras.preprocessing.image import load_img as im
from keras.preprocessing.image import save_img as sim
# import torch

In [4]:
import os
from random import shuffle
import numpy as np
import splitfolders
from PIL import Image
from torchvision import transforms
import numpy

In [5]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers,models
import wandb

In [6]:
src_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
src_zip = "nature_12K.zip"

In [7]:
%%capture
!curl -SL $src_url > $src_zip
!unzip $src_zip

In [8]:
data_folders="/content/inaturalist_12K/train"

In [9]:
splitfolders.ratio(data_folders, output="outputfin", seed=1337, ratio=(.9, .1), group_prefix=None)

Copying files: 9999 files [00:11, 855.67 files/s]


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!rm -rf /content/drive/MyDrive/outputfin

In [13]:
!cp -r /content/outputfin /content/drive/MyDrive/

In [ ]:
!rm -rf nature_12K.zip

In [ ]:
!rm -rf inaturalist_12K/train

In [14]:
class_labels = os.listdir("/content/drive/MyDrive/outputfin/train")

In [15]:
lab={l:i for l,i in zip(class_labels,range(10))}
lab

{'Amphibia': 9,
 'Animalia': 0,
 'Arachnida': 5,
 'Aves': 2,
 'Fungi': 3,
 'Insecta': 8,
 'Mammalia': 4,
 'Mollusca': 6,
 'Plantae': 1,
 'Reptilia': 7}

In [16]:
data_augmentation  = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [17]:
imgs_tr = []
filenames = []

truth_tr = []
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join("/content/drive/MyDrive/outputfin/train", l))
  for img in imgs_per_class:
    
    filenames.append(img.split(".")[0])
    
    img_path = os.path.join("/content/drive/MyDrive/outputfin/train", l, img)
    # img = image.load_img(img_path, target_size=(224, 224))
    # img = image.img_to_array(img)
    img=Image.open(img_path)
    img = img.resize((224,224))
    if img.mode == 'L':
                continue
    normalized_image = data_augmentation(img)
    truth_tr.append(lab[l])
    # don't forget to rescale test images to match the range of inputs
    # to the network
    # img = img/255.0
    imgs_tr.append(numpy.asarray(normalized_image).transpose(1,2,0))
x_train=np.asarray(imgs_tr)
y_train=np.asarray(truth_tr)
del imgs_tr
del truth_tr

In [22]:
x_train.shape

(8997, 224, 224, 3)

In [18]:
imgs_val = []
filenames = []

truth_val = []
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join("/content/drive/MyDrive/outputfin/val", l))
  for img in imgs_per_class:
    
    filenames.append(img.split(".")[0])
    
    img_path = os.path.join("/content/drive/MyDrive/outputfin/val", l, img)
    # img = image.load_img(img_path, target_size=(299, 299))
    # img = image.img_to_array(img)
    img=Image.open(img_path)
    img = img.resize((224,224))
    if img.mode == 'L':
                continue
    normalized_image = data_augmentation(img)
    truth_val.append(lab[l])
    # don't forget to rescale test images to match the range of inputs
    # to the network
    # img = img/255.0
    imgs_val.append(numpy.asarray(normalized_image).transpose(1,2,0))
x_val=np.asarray(imgs_val)
y_val=np.asarray(truth_val)
del imgs_val
del truth_val

In [21]:
x_val.shape

(999, 224, 224, 3)

In [19]:
imgs_val = []
filenames = []

truth_val = []
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join("/content/drive/MyDrive/inaturalist_12K/val", l))
  for img in imgs_per_class:
    
    filenames.append(img.split(".")[0])
    truth_val.append(lab[l])
    img_path = os.path.join("/content/drive/MyDrive/inaturalist_12K/val", l, img)
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    # don't forget to rescale test images to match the range of inputs
    # to the network
    img = img/255.0
    imgs_val.append(img)
x_test=np.asarray(imgs_val)
y_test=np.asarray(truth_val)
del imgs_val
del truth_val

In [43]:
import pathlib

In [45]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = pathlib.Path("/content/drive/MyDrive/outputfin/train"),
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      image_size = (224, 224),
                      shuffle = True,
                      seed = 17,
                      subset = 'training')

val_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = pathlib.path("/content/drive/MyDrive/outputfin/train"),
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      image_size = (224, 224),
                      shuffle = True,
                      seed = 17,
                      subset = 'validation')

ValueError: ignored

In [20]:
x_test.shape

(2000, 224, 224, 3)

In [ ]:
sweep_config = {
                  'method': 'grid',
                  'metric':{
                            'name':'accuracy',
                            'goal':'maximize'
                            },
                  'parameters': {
                                'filsize'   : {'values': [32, 64, 96, 128, 256]},
                                'kersize'   :{'values':[3,5,7,9]},
                                'dropout'   :{'values':[0.1,0.3,0.4]}, 
                                'denseunits':{'values':[256,100,50,60]}, 
                                'epochs'    :{'values':[5,10,20,30]},                          
                              }
              }

In [24]:
sweep_config = {
                  'method'    : "random",
                  'metric':{
                            'name':'val_accuracy',
                            'goal':'maximize'
                            },
                  'parameters': {
                                  'flt'                 : {'values' :[32, 64,16,]  },
                                  'filterorg'               : {'values' :[1,2,0.5] },
                                  'kerneldim'               : {'values' :[3,4,5,7]  },
                                  'activation'              : {'values' :['sigmoid','relu','tanh','elu'] },
                                  'neurons'                 : {'values' :[20,32,64,128,512] },
                                  'optimizer'               : {'values' :['sgd','adam','rmsprop'] },
                                  'lr'                      : {'values' :[1e-3,1e-4,2e-3,3e-3,4e-3,3e-4] },
                                  'epochs'                  : {'values' :[5] },
                                  'batchsize'               : {'values' :[16,32,64,128] },
                                  'dropout'                 : {'values' :[0.2,0.25,0.3,0,0.1,0.5] },
                                  'batchnormalisation'       : {'values' :['YES','NO']} ,
                                }
                }

In [ ]:
sweep_config

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'accuracy'},
 'parameters': {'denseunits': {'values': [256, 100, 50, 60]},
  'dropout': {'values': [0.1, 0.3, 0.4]},
  'epochs': {'values': [5, 10, 20, 30]},
  'filsize': {'values': [32, 64, 96, 128, 256]},
  'kersize': {'values': [3, 5, 7, 9]}}}

In [38]:
def create_CNN(fil,ker,activ,batchnormalisation,dropout,neurons):
  cnn=models.Sequential()
  cnn.add(layers.Conv2D(filters=fil[0], kernel_size=(ker[0], ker[0]), activation='relu',input_shape=(224, 224, 3)))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())
        
        
  cnn.add(layers.Conv2D(filters=fil[1], kernel_size=(ker[1], ker[1]), activation='relu'))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())
                                                  
  cnn.add(layers.Conv2D(filters=fil[2], kernel_size=(ker[2], ker[2]), activation='relu'))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())
         
  cnn.add(layers.Conv2D(filters=fil[3], kernel_size=(ker[3], ker[3]), activation='relu'))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())

  cnn.add(layers.Conv2D(filters=fil[4], kernel_size=(ker[4], ker[4]), activation='relu'))
  cnn.add(layers.MaxPooling2D((2, 2)))
  
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())


  cnn.add(layers.Flatten())
  
  cnn.add(layers.Dense(neurons, activation=activ))
  if(dropout!=0):
    cnn.add(layers.Dropout(dropout))
  cnn.add(layers.Dense(10, activation='softmax'))
  return cnn

In [ ]:
activation: relu
wandb: 	batchnormalisation: YES
wandb: 	batchsize: 128
wandb: 	dropout: 0.5
wandb: 	epochs: 5
wandb: 	filterorg: 0.5
wandb: 	flt: 16
wandb: 	kerneldim: 5
wandb: 	lr: 0.003
wandb: 	neurons: 20
wandb: 	optimizer: rmsprop

In [32]:
cnn=create_CNN([16,8,4,2,1],[5,5,5,5,5],'relu','YES',0.5,20)

In [ ]:
wandb: 	activation: relu
wandb: 	batchnormalisation: NO
wandb: 	batchsize: 16
wandb: 	dropout: 0.1
wandb: 	epochs: 5
wandb: 	filterorg: 2
wandb: 	flt: 32
wandb: 	kerneldim: 5
wandb: 	lr: 0.0003
wandb: 	neurons: 20
wandb: 	optimizer: rmsprop

In [39]:
cnn=create_CNN([32,64,128,256,512],[5,5,5,5,5],'relu','NO',0.1,20)

In [40]:
cnn.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.003),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
cnn.fit(x_train, y_train, batch_size=128,epochs=10,validation_data=(x_val,y_val))
  

AttributeError: ignored

In [33]:
def conlay():
  wandb.init(config=sweep_config, magic=True)  
  f= wandb.config.flt 
  fo=  wandb.config.filterorg                           
  fil=[ (int)(f*(fo)**i) if((int)(f*(fo)**i)>0) else  1 for i in range(5)]
  kd=wandb.config.kerneldim
  ker=[kd for i in range(5)]
  if(wandb.config.activation=="sigmoid"):
    activ=tf.keras.activations.sigmoid
  elif(wandb.config.activation=="tanh"):
    activ=tf.keras.activations.tanh
  elif(wandb.config.activation=="relu"):
    activ='relu'
  else:
    activ=tf.keras.activations.elu
  if(wandb.config.optimizer=="sgd"):
    optim = tf.keras.optimizers.SGD(learning_rate=wandb.config.lr)
  elif(wandb.config.optimizer=="adam"):
    optim = tf.keras.optimizers.Adam(learning_rate=wandb.config.lr)
  elif(wandb.config.optimizer=="momentum"):
    optim = tf.keras.optimizers.SGD(learning_rate=wandb.config.lr,momentum=wandb.config.gama)
  elif(wandb.config.optimizer=="rmsprop"):
    optim = tf.keras.optimizers.RMSprop(learning_rate=wandb.config.lr)
      
  cnn =create_CNN(fil,ker,activ,wandb.config.batchnormalisation,wandb.config.dropout,wandb.config.neurons)             
  cnn.compile(optimizer=optim,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  cnn.fit(x_train, y_train, batch_size=wandb.config.batchsize,epochs=wandb.config.epochs,validation_data=(x_val,y_val),callbacks=[wandb.keras.WandbCallback()])
  del cnn


In [27]:
wandb.login()


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [28]:
sweep_id = wandb.sweep(sweep_config,project='A2-part-A1')
wandb.agent(sweep_id,conlay,project='A2-part-A1',count=20)

Create sweep with ID: pxiyc0v0
Sweep URL: https://wandb.ai/pratap49/A2-part-A1/sweeps/pxiyc0v0


wandb: Agent Starting Run: rd9m5c4v with config:
wandb: 	activation: sigmoid
wandb: 	batchnormalisation: NO
wandb: 	batchsize: 32
wandb: 	dropout: 0.5
wandb: 	epochs: 5
wandb: 	filterorg: 2
wandb: 	flt: 16
wandb: 	kerneldim: 4
wandb: 	lr: 0.003
wandb: 	neurons: 30
wandb: 	optimizer: rmsprop
wandb: Currently logged in as: pratap49 (use `wandb login --relogin` to force relogin)


Epoch 1/5
282/282 [==============================] - 21s 31ms/step - loss: 2.3343 - accuracy: 0.0973 - val_loss: 2.3053 - val_accuracy: 0.1001 - _timestamp: 1648838379.0000 - _runtime: 34.0000
Epoch 2/5
282/282 [==============================] - 8s 28ms/step - loss: 2.3064 - accuracy: 0.1001 - val_loss: 2.3077 - val_accuracy: 0.1001 - _timestamp: 1648838387.0000 - _runtime: 42.0000
Epoch 3/5
282/282 [==============================] - 8s 28ms/step - loss: 2.3073 - accuracy: 0.0959 - val_loss: 2.3045 - val_accuracy: 0.1001 - _timestamp: 1648838395.0000 - _runtime: 50.0000
Epoch 4/5
282/282 [==============================] - 8s 28ms/step - loss: 2.3088 - accuracy: 0.0960 - val_loss: 2.3039 - val_accuracy: 0.1001 - _timestamp: 1648838403.0000 - _runtime: 58.0000
Epoch 5/5
282/282 [==============================] - 8s 28ms/step - loss: 2.3065 - accuracy: 0.0939 - val_loss: 2.3056 - val_accuracy: 0.1001 - _timestamp: 1648838410.0000 - _runtime: 65.0000



accuracy,▅█▃▃▁
epoch,▁▃▅▆█
loss,█▁▁▂▁
val_accuracy,▁▁▁▁▁
val_loss,▄█▂▁▄
accuracy,0.09392
best_epoch,3
best_val_loss,2.30388
epoch,4
loss,2.30645
val_accuracy,0.1001


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ysiff628 with config:
wandb: 	activation: tanh
wandb: 	batchnormalisation: NO
wandb: 	batchsize: 16
wandb: 	dropout: 0.25
wandb: 	epochs: 5
wandb: 	filterorg: 2
wandb: 	flt: 16
wandb: 	kerneldim: 3
wandb: 	lr: 0.003
wandb: 	neurons: 30
wandb: 	optimizer: adam


...

Epoch 1/5
71/71 [==============================] - 7s 78ms/step - loss: 2.4737 - accuracy: 0.1193 - val_loss: 2.2540 - val_accuracy: 0.1371 - _timestamp: 1648839104.0000 - _runtime: 17.0000
Epoch 2/5
71/71 [==============================] - 5s 63ms/step - loss: 2.2712 - accuracy: 0.1576 - val_loss: 2.3004 - val_accuracy: 0.1271 - _timestamp: 1648839109.0000 - _runtime: 22.0000
Epoch 3/5
71/71 [==============================] - 5s 64ms/step - loss: 2.2215 - accuracy: 0.1736 - val_loss: 2.2168 - val_accuracy: 0.1792 - _timestamp: 1648839113.0000 - _runtime: 26.0000
Epoch 4/5
71/71 [==============================] - 5s 65ms/step - loss: 2.2006 - accuracy: 0.1817 - val_loss: 2.2070 - val_accuracy: 0.1862 - _timestamp: 1648839118.0000 - _runtime: 31.0000
Epoch 5/5
71/71 [==============================] - 5s 65ms/step - loss: 2.1884 - accuracy: 0.1911 - val_loss: 2.1754 - val_accuracy: 0.2192 - _timestamp: 1648839122.0000 - _runtime: 35.0000



accuracy,▁▅▆▇█
epoch,▁▃▅▆█
loss,█▃▂▁▁
val_accuracy,▂▁▅▅█
val_loss,▅█▃▃▁
accuracy,0.19106
best_epoch,4
best_val_loss,2.17538
epoch,4
loss,2.18844
val_accuracy,0.21922


wandb: Agent Starting Run: msz5pwhh with config:
wandb: 	activation: tanh
wandb: 	batchnormalisation: NO
wandb: 	batchsize: 128
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	filterorg: 1
wandb: 	flt: 16
wandb: 	kerneldim: 5
wandb: 	lr: 0.004
wandb: 	neurons: 20
wandb: 	optimizer: rmsprop


Run msz5pwhh errored: InternalError()
wandb: ERROR Run msz5pwhh errored: InternalError()
wandb: Agent Starting Run: mysjd0i0 with config:
wandb: 	activation: sigmoid
wandb: 	batchnormalisation: NO
wandb: 	batchsize: 16
wandb: 	dropout: 0.25
wandb: 	epochs: 5
wandb: 	filterorg: 0.5
wandb: 	flt: 32
wandb: 	kerneldim: 3
wandb: 	lr: 0.004
wandb: 	neurons: 30
wandb: 	optimizer: adam


Run mysjd0i0 errored: InternalError()
wandb: ERROR Run mysjd0i0 errored: InternalError()


In [46]:
def conlayss():
  # wandb.init(config=sweep_config, magic=True)    
  cnn = models.Sequential([
                     
    layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64,  kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,  kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,  kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dropout(0.2),

    
    layers.Dense(30, activation='relu'),
    layers.Dense(10, activation='softmax')
  ])
  #gc.collect()
  # heyGPU=torch.cuda.is_available()
  # if heyGPU: cnnModel=cnnModel.cuda()
  train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
  train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

    # Prepare the validation dataset.
  val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
  val_dataset = val_dataset.batch(64)
  cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  cnn.fit(train_dataset,val_dataset,batch_size=64 ,epochs=10)
  del cnn
  # torch.cuda.empty_cache()

In [ ]:
y_train[5]

0

In [47]:
conlayss()

InternalError: ignored